# RunnableBranch
`RunnableBranch` 는 입력에 따라 동적으로 로직을 라우팅할 수 있는 강력한 도구입니다. 이를 통해 개발자는 입력 데이터의 특성에 기반하여 다양한 처리 경로를 유연하게 정의 할 수 있습니다.

`RunnableBranch` 는 복잡한 의사 결정 트리를 간단하고 직관적인 방식으로 구현할 수 있도록 도와줍니다. 이는 코드의 가독성과 유지보수성을 크게 향상시키며, 로직의 모듈화와 재사용성을 촉진합니다.

또한, `RunnableBranch` 는 런타임에 동적으로 분기 조건을 평가하고 적절한 처리 루틴을 선택할 수 있어, 시스템의 적응력과 확장성을 높여줍니다.

이러한 특징들로 인해 RunnableBranch는 다양한 도메인에서 활용될 수 있으며, 특히 입력 데이터의 다양성과 변동성이 큰 애플리케이션 개발에 매우 유용합니다. RunnableBranch 를 효과적으로 활용하면 코드의 복잡성을 줄이고, 시스템의 유연성과 성능을 향상시킬 수 있습니다.

# 입력에 따른 동적 로직 라우팅
LangChain Expression Language에서 라우팅을 수행하는 방법에 대해 다룹니다.

** 수행 방법 **
1. topic 추출하기 위한 chain을 만듭니다.
2. 각 topic에 대한 prompt chain을 만듭니다.
3. topic에 따라 chain을 반환하는 `route` 함수를 만듭니다.


라우팅을 통해 이전 단계의 출력이 다음 단계를 정의하는 비결정적 체인을 생성할 수 있습니다. 라우팅은 LLM과의 상호 작용에 구조와 일관성을 제공하는 데 도움이 됩니다.

라우팅을 수행하는 방법에는 두 가지가 있습니다.

1. `RunnableLambda` 에서 조건부로 실행 가능한 객체를 반환 (권장)
2. `RunnableBranch` 사용
두 가지 방법 모두 첫 번째 단계에서 입력 질문을 수학, 과학 또는 기타에 대한 것으로 분류한 다음, 해당 프롬프트 체인으로 라우팅하는 두 단계 시퀀스를 사용하여 설명하겠습니다.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model='gpt-4o-mini')

prompt = PromptTemplate.from_template(
    """
    주어진 사용자 질문을 '수학', '과학' 또는 '기타' 중 하나로 분류하세요. 한단어 이상으로 응답하지 마세요

#<question>
{question}
</question>

Classification:
"""
)

chain = (prompt | llm | StrOutputParser())

In [5]:
chain.invoke({"question": "2+2 는 무엇인가요?"})

'수학'

In [6]:
chain.invoke({"question": "작용 반작용의 법칙은 무엇인가요?"})

'과학'

In [7]:
chain.invoke({"question": "Google은 어떤 회사인가요?"})

'기타'

## RunnableLambda를 통한 routing

In [8]:
math_chain = (
    PromptTemplate.from_template(
        """You are an expert in math. \
Always answer questions starting with "차승현 선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | llm
)

science_chain = (
    PromptTemplate.from_template(
        """You are an expert in science. \
Always answer questions starting with "아이작 뉴턴 선생님께서 말씀하시기를..". \
Respond to the following question:

Question: {question}
Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | llm
)

general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question concisely:

Question: {question}
Answer:"""
    )
    # OpenAI의 LLM을 사용합니다.
    | llm
)

In [9]:
def route(info):
    # 주제에 "수학"이 포함되어 있는 경우
    if "수학" in info["topic"].lower():
        # datascience_chain을 반환
        return math_chain
    # 주제에 "과학"이 포함되어 있는 경우
    elif "과학" in info["topic"].lower():
        # art_chain을 반환
        return science_chain
    # 그 외의 경우
    else:
        # general_chain을 반환
        return general_chain

In [10]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda

full_chain = (
    {"topic": chain, "question": itemgetter("question")}
    | RunnableLambda(
        route
    )
    | StrOutputParser()
)

In [11]:
# 수학과 관련된 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."})

'차승현 선생님께서 말씀하시기를, 미적분은 수학의 한 분야로, 주로 변화와 관련된 문제를 다룹니다. 미적분학의 두 주요 구성 요소는 미분과 적분입니다. \n\n미분은 함수의 변화율을 측정하는 도구로, 특정 점에서 함수의 기울기를 구하는 방법입니다. 예를 들어, 물체의 속도를 구할 때 미분을 사용하여 위치의 변화율을 알아낼 수 있습니다.\n\n반면, 적분은 함수의 면적을 구하는 방법으로, 주어진 구간에서의 총합이나 누적된 양을 계산하는 데 사용됩니다. 예를 들어, 시간에 따른 속도를 알고 있을 때, 적분을 통해 이동한 총 거리를 계산할 수 있습니다.\n\n미적분은 물리학, 공학, 경제학 등 다양한 분야에서 널리 활용되며, 복잡한 문제를 해결하는 데 필수적인 도구입니다.'

In [12]:
# 과학과 관련된 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "중력은 어떻게 작용하나요?"})

'아이작 뉴턴 선생님께서 말씀하시기를, 중력은 두 물체 사이의 인력으로, 그들의 질량에 비례하고 거리의 제곱에 반비례한다고 하셨습니다. 즉, 질량이 클수록 더 강한 중력을 발휘하며, 물체 간의 거리가 멀어질수록 중력의 힘은 약해집니다. 뉴턴의 만유인력 법칙에 따르면, 모든 물체는 서로 중력을 통해 끌어당기며, 이는 우주에서의 운동과 행성의 궤도 형성에 중요한 역할을 합니다.'

In [13]:
# 기타 질문을 입력하여 체인을 호출합니다.
full_chain.invoke({"question": "RAG(Retrieval Augmented Generation)은 무엇인가요?"})

'RAG(정보 검색 기반 생성)는 자연어 처리 모델로, 대규모 데이터베이스에서 관련 정보를 검색한 후 이를 바탕으로 텍스트를 생성하는 방식입니다. 이 방법은 정보의 정확성을 높이고, 보다 풍부한 콘텐츠를 제공하기 위해 사용됩니다.'

## RunnableBranch 통한 routing

`RunnableBranch`는 입력값에 따라 실행할 조건과 `Runnable`을 정의할 수 있는 특별한 유형의 Runnable 입니다.

다만, 위에서 설명한 사용자 정의 함수로 구현할 수 없는 기능을 제공하지는 않으므로, 사용자 정의 함수를 사용하는 것이 좋습니다.

**문법**

- RunnableBranch는 (조건, Runnable) 쌍의 리스트와 기본 Runnable로 초기화됩니다.
- 호출 시 전달된 입력값을 각 조건에 전달하여 분기를 선택합니다.
- True로 평가되는 첫 번째 조건을 선택하고, 해당 조건에 해당하는 Runnable을 입력값과 함께 실행합니다.
- 제공된 조건과 일치하는 것이 없으면 기본 Runnable 을 실행합니다.

In [15]:
from operator import itemgetter
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    # 주제에 "수학"이 포함되어 있는지 확인하고, 포함되어 있다면 math_chain을 실행합니다.
    (lambda x: "수학" in x["topic"].lower(), math_chain),
    # 주제에 "과학"이 포함되어 있는지 확인하고, 포함되어 있다면 science_chain을 실행합니다.
    (lambda x: "과학" in x["topic"].lower(), science_chain),
    # 위의 조건에 해당하지 않는 경우 general_chain을 실행합니다.
    general_chain,
)
# 주제와 질문을 입력받아 branch를 실행하는 전체 체인을 정의합니다.
full_chain = (
    {"topic": chain, "question": itemgetter("question")} | branch | StrOutputParser()
)

In [16]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."})

'차승현 선생님께서 말씀하시기를, 미적분은 수학의 한 분야로, 크게 미분과 적분으로 나눌 수 있습니다. 미분은 함수의 변화율을 다루며, 주어진 함수의 기울기를 계산하는 데 사용됩니다. 이는 특정 지점에서의 함수의 순간적인 변화량을 나타냅니다. 반면, 적분은 함수의 면적을 구하거나, 누적량을 계산하는 데 사용됩니다. 두 개념은 서로 깊은 관계가 있으며, 미적분학의 기본 정리는 미분과 적분이 서로 반대되는 과정을 설명해 줍니다. 이러한 개념들은 자연과학, 공학, 경제학 등 다양한 분야에서 매우 중요한 역할을 합니다.'

In [17]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "중력 가속도는 어떻게 계산하나요?"})

'아이작 뉴턴 선생님께서 말씀하시기를, 중력 가속도는 물체가 중력의 영향을 받아 떨어질 때의 가속도를 의미하며, 지구 표면에서의 평균 값은 약 9.81 m/s²입니다. 이를 계산하기 위해서는 뉴턴의 만유인력 법칙을 사용합니다. \n\n만유인력 법칙에 따르면, 두 물체 간의 중력은 다음과 같은 공식으로 표현됩니다:\n\n\\[ F = \\frac{G \\cdot m_1 \\cdot m_2}{r^2} \\]\n\n여기서 \\( F \\)는 중력, \\( G \\)는 만유인력 상수, \\( m_1 \\)과 \\( m_2 \\)는 두 물체의 질량, \\( r \\)은 두 물체 간의 거리입니다. \n\n지구의 경우, 지구의 질량 \\( M \\)과 반지름 \\( R \\)을 고려할 때, 지구 표면에서의 중력 가속도 \\( g \\)는 다음과 같이 표현할 수 있습니다:\n\n\\[ g = \\frac{G \\cdot M}{R^2} \\]\n\n이 공식을 통해 중력 가속도를 계산할 수 있으며, 실험적으로도 측정할 수 있습니다.'

In [18]:
# 질문을 입력하여 전체 체인을 실행합니다.
full_chain.invoke({"question": "RAG(Retrieval Augmented Generation)은 무엇인가요?"})

'RAG(정보 검색 증강 생성)는 정보 검색과 자연어 생성 기술을 결합한 모델로, 외부 데이터베이스에서 정보를 검색한 후 이를 활용하여 더 정확하고 관련성 높은 응답을 생성하는 방식입니다. 이 접근법은 대화형 AI 및 질의응답 시스템에서 효과적으로 사용됩니다.'